## Text input

https://platform.openai.com/docs/models

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.agents import create_agent

agent = create_agent(
    model='gpt-5-nano',
    system_prompt="You are a science fiction writer, create a capital city at the users request.",
)

In [3]:
from langchain.messages import HumanMessage

question = HumanMessage(content=[
    {"type": "text", "text": "What is the capital of The Moon?"}
])

response = agent.invoke(
    {"messages": [question]}
)

print(response['messages'][-1].content)

Capital: Lunaris Prime

Description: Lunaris Prime sits along the illuminated rim of Shackleton Crater near the Moon’s south pole. The city is a layered, glass-and-titanium metropolis carved into ice and basalt, with a network of transit tunnels and magnetic lev trains weaving between domes. Tall spires and crystalline towers catch the scarce sunlight and reflect it across the ice-field, while the Lantern Belt—rings of solar collectors and water-processing stations—keeps the city powered and watered. The lunar seat of governance, the Lunar Council, meets in the Beacon Spire at the city’s heart, guiding a federation of lunar settlements and Earth-mined colonies. If you’d like, I can tailor its culture, architecture, or political structure to a specific sci-fi setting.


## Image input

In [4]:
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept='.png', multiple=False)
display(uploader)

FileUpload(value=(), accept='.png', description='Upload')

In [5]:
print(uploader.value)

()


In [7]:
import base64

# Get the first (and only) uploaded file dict
uploaded_file = uploader.value[0]

# This is a memoryview
content_mv = uploaded_file["content"]

# Convert memoryview -> bytes
img_bytes = bytes(content_mv)  # or content_mv.tobytes()

# Now base64 encode
img_b64 = base64.b64encode(img_bytes).decode("utf-8")

In [8]:
multimodal_question = HumanMessage(content=[
    {"type": "text", "text": "Tell me about this capital"},
    {"type": "image", "base64": img_b64, "mime_type": "image/png"}
])

response = agent.invoke(
    {"messages": [multimodal_question]}
)

print(response['messages'][-1].content)

Here’s a capital built for a future that feels familiar and strange at once. Meet Lumenhaven, the radiant capital of the Solar Concord.

What it is and where it sits
- Lumenhaven sits on a crescent of coastline that wraps around a wind- and tide-driven bay. The city is carved into three living tiers: sea level, mid-height, and the apex of the skyline.
- Its heart is the Crown Spire, a thousand-meter-tall tower that glows with bioluminescent algae farms, solar sails, and adaptive glass that shifts tint with the time of day.

Layout and feel
- Sea Level Ring: The harbor districts, markets, and ports. Buildings here are squat and practical, wrapped in algae-green terraces that harvest energy and clean air.
- Mid Tier: Residential districts, schools, hospitals, and research campuses. Streets are lined with vertical gardens and modular balconies that lean toward the sun when needed.
- Apex District: The political core. The Crown Spire’s base houses the Solar Council,augmented reality plazas

## Audio input

In [14]:
import sounddevice as sd
from scipy.io.wavfile import write
import base64
import io
import time
from tqdm import tqdm

# Recording settings
duration = 5  # seconds
sample_rate = 44100

print("Recording...")
audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
# Progress bar for the duration
for _ in tqdm(range(duration * 10)):   # update 10× per second
    time.sleep(0.1)
sd.wait()
print("Done.")

# Write WAV to an in-memory buffer
buf = io.BytesIO()
write(buf, sample_rate, audio)
wav_bytes = buf.getvalue()

aud_b64 = base64.b64encode(wav_bytes).decode("utf-8")

Recording...


100%|██████████| 50/50 [00:05<00:00,  9.55it/s]


Done.


In [15]:
agent = create_agent(
    model='gpt-4o-audio-preview',
)

multimodal_question = HumanMessage(content=[
    {"type": "text", "text": "Tell me about this audio file"},
    {"type": "audio", "base64": aud_b64, "mime_type": "audio/wav"}
])

response = agent.invoke(
    {"messages": [multimodal_question]}
)

print(response['messages'][-1].content)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}